In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok config add-authtoken 336lPPsm0HsytCjCcrcZgatuNqs_7G4BjrWTsSPAGmP3hRpF9

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import pickle
import json

app = Flask(__name__)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Qwen2-VL model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    device_map="auto"
)

min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    min_pixels=min_pixels,
    max_pixels=max_pixels
)

# Load salary predictor model
with open("salary_predictor.pkl", "rb") as f:
    salary_model = pickle.load(f)

# Load label encoders
with open("label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)


@app.route("/predict", methods=["POST"])
def predict():
    try:
        if "image" not in request.files:
            return jsonify({"error": "No image uploaded"}), 400

        image_file = request.files["image"]
        image = Image.open(image_file.stream)

        prompt = """
          Read this CV image and extract structured job-related information in JSON format.
          The JSON MUST follow this structure:

          {
            "job_title": "string",
            "experience_level": "string",
            "employment_type": "string",
            "company_size": "string",
            "employee_residence": "string",
            "education_required": "string",
            "years_experience": "int",
            "industry": "string"
          }

          - Only output a valid JSON object (no explanations, no text outside JSON).
          """

        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt},
                ],
            }
        ]

        text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

        inputs = processor(
            text=[text_prompt],
            images=[image],
            padding=True,
            return_tensors="pt"
        ).to(device)

        output_ids = model.generate(**inputs, max_new_tokens=256)
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(inputs.input_ids, output_ids)
        ]

        output_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )

        # Ensure valid JSON
        try:
            job_data = json.loads(output_text[0])
        except json.JSONDecodeError:
            return jsonify({"error": "Model did not return valid JSON", "raw_output": output_text[0]}), 500

        # Apply label encoding to categorical fields
        features = []
        for field in [
            "job_title",
            "experience_level",
            "employment_type",
            "company_size",
            "employee_residence",
            "education_required",
            "industry"
        ]:
            value = job_data.get(field, "")
            if field in label_encoders:
                try:
                    encoded_val = label_encoders[field].transform([value])[0]
                except ValueError:
                    # unseen label → default to -1
                    encoded_val = -1
                features.append(encoded_val)
            else:
                features.append(-1)

        # Add numeric years_experience
        years_exp = job_data.get("years_experience", 0)
        try:
            years_exp = int(years_exp)
        except:
            years_exp = 0
        features.append(years_exp)

        # Predict salary
        predicted_salary = salary_model.predict([features])[0]

        # Add salary to response
        job_data["predicted_salary"] = int(predicted_salary)

        return jsonify(job_data)

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Start Flask
port = 5000
public_url = ngrok.connect(port)
print(f"🚀 Public URL: {public_url}")
app.run(port=port)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🚀 Public URL: NgrokTunnel: "https://shannon-admissive-nguyet.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 19:35:04] "POST /predict HTTP/1.1" 200 -


```json
[
  {
    "name": "Software Developer",
    "salary": 80000
  },
  {
    "name": "AI Engineer",
    "salary": 90000
  },
  {
    "name": "Data Scientist",
    "salary": 75000
  },
  {
    "name": "Machine Learning Engineer",
    "salary": 85000
  },
  {
    "name": "AI Architect",
    "salary": 95000
  }
]
```


INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 19:51:50] "POST /predict HTTP/1.1" 200 -


```json
[
  {
    "name": "Software Developer",
    "salary": 80000
  },
  {
    "name": "AI Engineer",
    "salary": 90000
  },
  {
    "name": "Data Scientist",
    "salary": 75000
  },
  {
    "name": "Machine Learning Engineer",
    "salary": 85000
  },
  {
    "name": "AI Architect",
    "salary": 95000
  }
]
```


In [8]:
import torch, gc

# Try to delete big variables if they exist
for var_name in ["model", "processor", "generated_ids", "output_text"]:
    if var_name in locals():
        del globals()[var_name]

# Force garbage collection
gc.collect()

# Clear PyTorch cache
torch.cuda.empty_cache()
